In [38]:
import math
import random

sc = SparkContext.getOrCreate()

TRAIN_TEST = 0.8

global n_columns
global w
global b
global n_row

n_columns=56 #total number of columns (58) - the one deleted (57) - label (56)
col_sums=[]
averages=[]
sigmas=[]
w=[]
b=[]

def initializeAccumulators():
    i=0
    while(i<n_columns):
        averages.append(sc.accumulator(0))
        col_sums.append(sc.accumulator(0))
        sigmas.append(sc.accumulator(0)) 
        i+=1

def addToAccumulators(row):
    if(len(row)!=len(col_sums)):
        raise Exception("Number of columns in the row doesn't mach the number of accumulators initiated. Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
    i=0
    while(i<n_columns):
        print("Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
        col_sums[i].add(row[i])
        i+=1
        
def preprocessing(row):
    #splitting
    split=row.split(" ")
    label = int(split[57])
    del split[57] #label
    del split[56] #col 57th
    split=[float(col) for col in split]
    addToAccumulators(split)
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    #return
    return (key,train,split,label)
def calcAvg(n_row):
    i=0
    while(i<n_columns):
        averages[i]=col_sums[i].value/n_row
        i+=1
def calcResiduals(row):
    i=0
    while(i<n_columns):       
        sigmas[i].add(math.pow(row[2][i]-averages[i],2))
        i+=1
    return row
def calcSigmas(n_row):
    i=0
    while(i<n_columns):
        sigmas[i]=math.sqrt(sigmas[i].value/float(n_row-1))
        i+=1
def normalize(row):
    i=0
    while(i<n_columns):
        row[2][i]=(row[2][i]-averages[i])/sigmas[i]
        i+=1
    return row
def initializeWeights(random_init=False):
    if(random_init):
        return [random.random() for i in range(0,n_columns)]
    else:
        return [0 for i in range(0,n_columns)]

def initializeBias():
    return [0 for i in range(0,n_columns)]

def predict(w,b,X):
    return w*X+b

def train(filename, iterations, learning_rate, lambda_reg):
    initializeAccumulators()
    dataset=sc.textFile(filename).map(preprocessing).sortBy(lambda x: x[0])
    n_row=dataset.count()
#    print(dataset.first())
    calcAvg(n_row)
    dataset.map(calcResiduals).collect()
    calcSigmas(n_row)
    dataset=dataset.map(normalize)
#    print(dataset.first())
    train=dataset.filter(lambda x: x[1]==1)
    train_x=train.map(lambda x: x[2])
    train_y=train.map(lambda x: x[3])
    test=dataset.filter(lambda x: x[1]==0)
    test_x=test.map(lambda x: x[2])
    test_y=test.map(lambda x: x[3])
    print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")
#    w=initializeWeights()
#    b=initializeBias()
#    print(predict(w,b,train_x.first()))
train("dataset/spam.data","a","a","a")

Split train/test done. Train contains 3718 elements, Test contains 883 elements


In [ ]:
def gradDescent(w,b,y_pred,y_train):

In [ ]:


X=preprocessing.normalize(dataset.drop("f57", axis=1), norm='l2')
y=dataset["f57"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)